In [ ]:
from IPython.display import clear_output

In [ ]:
!pip install pycodestyle pycodestyle_magic
!pip install flake8
clear_output()

In [ ]:
%load_ext pycodestyle_magic

In [ ]:
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
from tqdm.notebook import tqdm
from string import ascii_lowercase
from datetime import datetime, timedelta

%matplotlib inline

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
plt.switch_backend('agg')

# Data preparation

In [ ]:
df = pd.read_csv('rus.txt', sep='\t', header=None).drop(2, axis=1)

In [ ]:
df.head(15)

,0,1
0,Go.,Марш!
1,Go.,Иди.
2,Go.,Идите.
3,Hi.,Здравствуйте.
4,Hi.,Привет!
5,Hi.,Хай.
6,Hi.,Здрасте.
7,Hi.,Здоро́во!
8,Run!,Беги!
9,Run!,Бегите!


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 414010 entries, 0 to 414009
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   0       414010 non-null  object
 1   1       414010 non-null  object
dtypes: object(2)
memory usage: 6.3+ MB


In [ ]:
!rm *.txt

In [ ]:
df[0] = df[0].apply(lambda x: re.sub(r'[^a-zа-яё ]+', '', x.lower()))
df[1] = df[1].apply(lambda x: re.sub(r'[^a-zа-яё ]+', '', x.lower()))
df[0] = df[0].apply(lambda x: re.sub(r'\s\s+', ' ', x))
df[1] = df[1].apply(lambda x: re.sub(r'\s\s+', ' ', x))

# по-хорошему ещё надо бы лемматизацию сделать, но ладно, здесь не те данные

In [ ]:
df.sample(n=15)

,0,1
401122,the prince fell in love with a woodcutters dau...,принц влюбился в дочь лесоруба
344036,oranges are graded by size and quality,апельсины сортируют по величине и по качеству
156451,all of my things are gone,все мои вещи пропали
216748,never keep food in your tent,никогда не храните пищу в палатке
174865,do you want some more cake,хочешь ещё торта
237193,tom needs to be there tonight,тому нужно там вечером быть
15723,i speak french,я владею французским языком
323101,i dont see tom as much as i used to,я тома не так часто вижу как раньше
284268,tom i dont want to talk to you,том я не хочу с тобой разговаривать
218801,this watch needs to be fixed,эти часы нужно починить


Максимальную длину будем выбирать исходя из средней длины фразы (в символах, делить на слова при таких коротких фразах нет особого смысла).

In [ ]:
np.mean([len(x) for x in df[0]] + [len(x) for x in df[1]])

28.36894760996111

In [ ]:
# не забудем про символы начала и конца строки

MAX_LEN = 28

In [ ]:
pairs = list(zip(df[0].tolist(), df[1].tolist()))

In [ ]:
pairs[np.random.randint(414011)]

('im trying to rid myself of this bad habit',
 'я пытаюсь избавиться от этой плохой привычки')

In [ ]:
del df

In [ ]:
english_lowercase = ['SOS', 'EOS', ' '] + list(ascii_lowercase)

In [ ]:
print(english_lowercase)

['SOS', 'EOS', ' ', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [ ]:
# в предложениях на русском встречается латиница, например, в слове 'ipad'

russian_lowercase = ['SOS', 'EOS', ' '] + list(
    'абвгдеёжзийклмнопрстуфхцчшщьыъэюя') + list(ascii_lowercase)

In [ ]:
print(russian_lowercase)

['SOS', 'EOS', ' ', 'а', 'б', 'в', 'г', 'д', 'е', 'ё', 'ж', 'з', 'и', 'й', 'к', 'л', 'м', 'н', 'о', 'п', 'р', 'с', 'т', 'у', 'ф', 'х', 'ц', 'ч', 'ш', 'щ', 'ь', 'ы', 'ъ', 'э', 'ю', 'я', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [ ]:
eng_to_idx = {elem: i for i, elem in enumerate(english_lowercase)}
rus_to_idx = {elem: i for i, elem in enumerate(russian_lowercase)}

In [ ]:
def tensors_from_pair(pair):
    eng_idx = [0] + [eng_to_idx[letter] for letter in pair[0][:MAX_LEN]] + [1]
    rus_idx = [0] + [rus_to_idx[letter] for letter in pair[1][:MAX_LEN]] + [1]
    eng_tensor = torch.tensor(eng_idx, dtype=torch.long,
                              device=device).view(-1, 1)
    rus_tensor = torch.tensor(rus_idx, dtype=torch.long,
                              device=device).view(-1, 1)
    return (eng_tensor, rus_tensor)

# Encoder
Добавим параметры 'rnn_type' и 'rnn_n', чтобы не переписывать несколько раз один и тот же код.

In [ ]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size, rnn_type='gru', rnn_n=1):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.rnn_n = rnn_n

        self.embedding = nn.Embedding(input_size, hidden_size)
        if rnn_type == 'gru':
            self.rnn = nn.GRU(hidden_size, hidden_size)
        elif rnn_type == 'lstm':
            self.rnn = nn.LSTM(hidden_size, hidden_size)
        if self.rnn_n == 2:
            if rnn_type == 'gru':
                self.rnn_2 = nn.GRU(hidden_size, hidden_size)
            elif rnn_type == 'lstm':
                self.rnn_2 = nn.LSTM(hidden_size, hidden_size)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output, hidden = self.rnn(output, hidden)
        if self.rnn_n == 2:
            output, hidden = self.rnn_2(output, hidden)
        return output, hidden

    def initHidden(self):
        if isinstance(self.rnn, nn.LSTM):
            return (torch.zeros(1, 1, self.hidden_size, device=device),
                    torch.zeros(1, 1, self.hidden_size, device=device))
        return torch.zeros(1, 1, self.hidden_size, device=device)

# Decoder
Аналогично добавляем параметры `rnn_type` и `rnn_n`.

In [ ]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, rnn_type='gru', rnn_n=1):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.rnn_n = rnn_n

        self.embedding = nn.Embedding(output_size, hidden_size)
        if rnn_type == 'gru':
            self.rnn = nn.GRU(hidden_size, hidden_size)
        elif rnn_type == 'lstm':
            self.rnn = nn.LSTM(hidden_size, hidden_size)
        if self.rnn_n == 2:
            if rnn_type == 'gru':
                self.rnn_2 = nn.GRU(hidden_size, hidden_size)
            elif rnn_type == 'lstm':
                self.rnn_2 = nn.LSTM(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.rnn(output, hidden)
        if self.rnn_n == 2:
            output, hidden = self.rnn_2(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        if isinstance(self.rnn, nn.LSTM):
            return (torch.zeros(1, 1, self.hidden_size, device=device),
                    torch.zeros(1, 1, self.hidden_size, device=device))
        return torch.zeros(1, 1, self.hidden_size, device=device)

# Функции для обучения и оценки качества

In [ ]:
teacher_forcing_ratio = 0.5

Уберём кусок с `use_teaching_forcing` в один цикл вместо двух.

In [ ]:
def train(input_tensor, target_tensor, encoder, decoder,
          encoder_optimizer, decoder_optimizer, criterion, max_len=MAX_LEN):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    encoder_outputs = torch.zeros(max_len + 2, encoder.hidden_size,
                                  device=device)
    loss = 0

    for i in range(input_tensor.size(0)):
        encoder_output, encoder_hidden = encoder(input_tensor[i],
                                                 encoder_hidden)
        encoder_outputs[i] = encoder_output[0, 0]

    decoder_input = torch.tensor([[0]], device=device)
    decoder_hidden = encoder_hidden

    if np.random.random() < teacher_forcing_ratio:
        use_teacher_forcing = True
    else:
        use_teacher_forcing = False

    for i in range(target_tensor.size(0)):
        decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
        loss += criterion(decoder_output, target_tensor[i])
        if use_teacher_forcing:
            # teacher forcing: feed the target as the next input
            decoder_input = target_tensor[i]
        else:
            # use its own predictions as the next input
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()
            if decoder_input.item() == 1:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_tensor.size(0)

Я убрала все мелкие вспомогательные функции (имхо, ненужное усложнение) в функцию `train_epochs`.

In [ ]:
def train_epochs(encoder, decoder, n_epochs=75000, learning_rate=0.01):
    start = datetime.now()
    plot_losses = []
    print_loss = 0  # reset every 5000 epochs
    plot_loss = 0  # reset every 100 epochs

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensors_from_pair(pairs[np.random.randint(414011)])
                      for epoch in range(n_epochs)]
    criterion = nn.NLLLoss()

    for epoch in tqdm(range(n_epochs)):
        training_pair = training_pairs[epoch]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder, decoder,
                     encoder_optimizer, decoder_optimizer, criterion)
        print_loss += loss
        plot_loss += loss

        if (epoch + 1) % 5000 == 0:
            print_loss /= 5000
            delta = datetime.now() - start
            stats = f'''
            Epoch {epoch + 1} ({(epoch + 1) * 100 / n_epochs:.1f}%)
            Time: {delta.seconds // 60}m {delta.seconds % 60}s
            Loss: {print_loss:.4f}
            '''
            print(stats)
            print_loss = 0

        if (epoch + 1) % 100 == 0:
            plot_loss /= 100
            plot_losses.append(plot_loss)
            plot_loss = 0

    plt.figure(figsize=(14, 12))
    fig, ax = plt.subplots()
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(plot_losses)
    plt.show()

In [ ]:
def evaluate(encoder, decoder, sent, max_len=MAX_LEN):
    with torch.no_grad():
        idx = [0] + [eng_to_idx[letter] for letter in sent[:max_len]] + [1]
        input_tensor = torch.tensor(idx, dtype=torch.long,
                                    device=device).view(-1, 1)
        encoder_hidden = encoder.initHidden()
        encoder_outputs = torch.zeros(max_len + 2, encoder.hidden_size,
                                      device=device)

        for i in range(input_tensor.size()[0]):
            encoder_output, encoder_hidden = encoder(input_tensor[i],
                                                     encoder_hidden)
            encoder_outputs[i] += encoder_output[0, 0]

        decoder_input = torch.tensor([[0]], device=device)
        decoder_hidden = encoder_hidden
        decoded_letters = ''

        for i in range(max_len):
            decoder_output, decoder_hidden = decoder(decoder_input,
                                                     decoder_hidden)
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == 1:
                break
            else:
                decoded_letters += russian_lowercase[topi.item()]
            decoder_input = topi.squeeze().detach()

        return decoded_letters

In [ ]:
def evaluate_random(encoder, decoder, n=10):
    for i in range(n):
        pair = pairs[np.random.randint(414011)]
        res = f'''
        Eng:  {pair[0]}
        Rus:  {pair[1]}
        Pred: {evaluate(encoder, decoder, pair[0])[3:]}
        '''
        # SOS в начале нам не нужен
        print(res)

# Обучение
## GRU с одним рекуррентным слоем

In [ ]:
hidden_size = 256

In [ ]:
encoder = EncoderRNN(len(english_lowercase), hidden_size).to(device)
decoder = DecoderRNN(hidden_size, len(russian_lowercase)).to(device)

In [ ]:
train_epochs(encoder, decoder)


            Epoch 5000 (6.7%)
            Time: 3m 48s
            Loss: 2.5566
            

            Epoch 10000 (13.3%)
            Time: 7m 31s
            Loss: 2.3997
            

            Epoch 15000 (20.0%)
            Time: 11m 12s
            Loss: 2.3262
            

            Epoch 20000 (26.7%)
            Time: 14m 52s
            Loss: 2.2594
            

            Epoch 25000 (33.3%)
            Time: 18m 32s
            Loss: 2.2369
            

            Epoch 30000 (40.0%)
            Time: 22m 15s
            Loss: 2.2119
            

            Epoch 35000 (46.7%)
            Time: 25m 56s
            Loss: 2.1656
            

            Epoch 40000 (53.3%)
            Time: 29m 38s
            Loss: 2.1301
            

            Epoch 45000 (60.0%)
            Time: 33m 18s
            Loss: 2.1213
            

            Epoch 50000 (66.7%)
            Time: 36m 58s
            Loss: 2.0822
            

            Epoch 55000 (73.3%)
 

Финальный loss – 2.0378. Посмотрим на генерацию предложений.

In [ ]:
evaluate_random(encoder, decoder)


        Eng:  drink some tea
        Rus:  выпей чаю
        Pred: перед о    о
        

        Eng:  wherever you go ill follow
        Rus:  куда бы ты ни пошёл я пойду следом
        Pred: где ты вышел получает
        

        Eng:  id like to play tennis
        Rus:  я хотел бы играть в теннис
        Pred: я хотел бы от то         
        

        Eng:  this book was printed in england
        Rus:  эту книгу напечатали в англии
        Pred: это был                 
        

        Eng:  tom can answer
        Rus:  том может ответить
        Pred: том может по тто от 
        

        Eng:  this dictionary has about entries
        Rus:  в этом словаре примерно статей
        Pred: это оте                  
        

        Eng:  we cant trust tom
        Rus:  мы не можем доверять тому
        Pred: мы не можем по тоото тот 
        

        Eng:  ill call you when im ready
        Rus:  я позвоню вам когда буду готова
        Pred: я могу тебе  т          
       

Ну, предсказания даже на что-то похожи!
## GRU с двумя рекуррентными слоями

In [ ]:
encoder = EncoderRNN(len(english_lowercase), hidden_size, rnn_n=2).to(device)
decoder = DecoderRNN(hidden_size, len(russian_lowercase), rnn_n=2).to(device)

In [ ]:
train_epochs(encoder, decoder)


            Epoch 5000 (6.7%)
            Time: 5m 57s
            Loss: 2.5335
            

            Epoch 10000 (13.3%)
            Time: 11m 53s
            Loss: 2.3725
            

            Epoch 15000 (20.0%)
            Time: 17m 51s
            Loss: 2.3109
            

            Epoch 20000 (26.7%)
            Time: 23m 45s
            Loss: 2.2585
            

            Epoch 25000 (33.3%)
            Time: 29m 42s
            Loss: 2.2422
            

            Epoch 30000 (40.0%)
            Time: 35m 39s
            Loss: 2.2372
            

            Epoch 35000 (46.7%)
            Time: 41m 38s
            Loss: 2.2261
            

            Epoch 40000 (53.3%)
            Time: 47m 34s
            Loss: 2.2062
            

            Epoch 45000 (60.0%)
            Time: 53m 33s
            Loss: 2.2031
            

            Epoch 50000 (66.7%)
            Time: 59m 33s
            Loss: 2.1752
            

            Epoch 55000 (73.3%)


От добавления ещё одного рекуррентного слоя лосс не только не улучшился, а даже наоборот (финальный лосс – 2.1504). :( Посмотрим на сгенерированные предложения.

In [ ]:
evaluate_random(encoder, decoder)


        Eng:  a stranger spoke to me
        Rus:  со мной заговорил незнакомец
        Pred: кото то               
        

        Eng:  theyre both dead
        Rus:  они оба умерли
        Pred: они отере               
        

        Eng:  i cant see who tom is talking to
        Rus:  я не вижу с кем говорит том
        Pred: я не                       
        

        Eng:  were you scared
        Rus:  вы испугались
        Pred: ты не            
        

        Eng:  what makes us special
        Rus:  что делает нас особенными
        Pred: ты  ео                  
        

        Eng:  he was beside himself with joy
        Rus:  он был вне себя от радости
        Pred: они отец н                 
        

        Eng:  this is your big chance
        Rus:  это ваш шанс каких больше не будет
        Pred: это нашине с      
        

        Eng:  did you notify tom
        Rus:  ты уведомил тома
        Pred: ты  ое            
        

        Eng:  are you 

Сгенерированные предложения тоже хуже. Посмотрим, что даст LSTM с одним рекуррентным слоем.
## LSTM c одним рекуррентным слоем

In [ ]:
encoder = EncoderRNN(
    len(english_lowercase), hidden_size, rnn_type='lstm').to(device)
decoder = DecoderRNN(
    hidden_size, len(russian_lowercase), rnn_type='lstm').to(device)

In [ ]:
train_epochs(encoder, decoder)


            Epoch 5000 (6.7%)
            Time: 4m 2s
            Loss: 2.5078
            

            Epoch 10000 (13.3%)
            Time: 8m 1s
            Loss: 2.3721
            

            Epoch 15000 (20.0%)
            Time: 11m 58s
            Loss: 2.3341
            

            Epoch 20000 (26.7%)
            Time: 15m 57s
            Loss: 2.3031
            

            Epoch 25000 (33.3%)
            Time: 19m 57s
            Loss: 2.2486
            

            Epoch 30000 (40.0%)
            Time: 23m 55s
            Loss: 2.2347
            

            Epoch 35000 (46.7%)
            Time: 27m 52s
            Loss: 2.1913
            

            Epoch 40000 (53.3%)
            Time: 31m 49s
            Loss: 2.1657
            

            Epoch 45000 (60.0%)
            Time: 35m 45s
            Loss: 2.1597
            

            Epoch 50000 (66.7%)
            Time: 39m 48s
            Loss: 2.1322
            

            Epoch 55000 (73.3%)
   

Финальный лосс чуть выше, чем у GRU (2.0566). Посмотрим на генерацию предложений.

In [ ]:
evaluate_random(encoder, decoder)


        Eng:  i dont know what tom was doing
        Rus:  я не знаю чем том занимался
        Pred: я не знаю что тто          
        

        Eng:  there are millions of stars in the universe
        Rus:  во вселенной миллионы звезд
        Pred: давайте на                 
        

        Eng:  can you break a five dollar bill
        Rus:  ты пять долларов не разменяешь
        Pred: мы не                      
        

        Eng:  it happened three years later that is in 
        Rus:  это произошло три года спустя то есть в году
        Pred: это были последним со      
        

        Eng:  please speak a little slower
        Rus:  говори немного помедленнее пожалуйста
        Pred: послен                     
        

        Eng:  tom is in the warehouse
        Rus:  том на складе
        Pred: том пол                
        

        Eng:  we deserve better
        Rus:  мы заслуживаем лучшего
        Pred: мы не            
        

        Eng:  its not far

Примерно на том же уровне, что и у GRU.